# Turn Restrictions
This notebook is for donwloading OSM's turn restriction data which are stored as relations.

In [ ]:
import requests
import json
from pathlib import Path 
import geopandas as gpd
import pandas as pd

import sys
path = Path.cwd().parent
sys.path.insert(0,str(path))
import file_structure_setup
config = file_structure_setup.filepaths(path)

In [ ]:
fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot/networks'

bikewaysim_studyarea = gpd.read_file(Path.home()/'Documents/BikewaySimData/Data/Study Areas/bikewaysim_studyarea.geojson')

links = gpd.read_file(fp/'reconciled_network.gpkg',layer='links_w_signals_elevation',mask=bikewaysim_studyarea)
links.to_crs('epsg:4326',inplace=True)

minx, miny, maxx, maxy = links.total_bounds

# Query

In [ ]:
query = f"""
    [out:json]
    [timeout:30]
    ;
    (relation
        ["restriction"="no_u_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_left_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="only_straight_on"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_right_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="only_left_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_straight_on"]
        ({miny},{minx},{maxy},{maxx});
    );
    out geom;
    """

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
result = r.json()

Desired format is: 
| Relation ID | From linkid | To linkid | restriction type |
| --- | --- | --- | --- |
| test | test | test | test |

In [ ]:
turn_restrictions = []

for element in result['elements']:
    relation_id = element['id']
    restriction = element['tags']['restriction']

    #there can be multiple to restrictions
    to_way_ids = []

    for member in element['members']:
        if member['type'] == 'way':
            if member['role'] == 'from':
                from_way_id = member['ref']
            else:
                to_way_ids.append(member['ref'])

    turn_restrictions.append({
        'relation_id': relation_id,
        'restriction': restriction,
        'from_way_id': from_way_id,
        'to_way_id': to_way_ids
    })

turn_restrictions = pd.DataFrame().from_dict(turn_restrictions).explode('to_way_id')
turn_restrictions


In [ ]:
turn_restrictions['type'] = turn_restrictions['restriction'].apply(lambda text: text.split('_')[0])
turn_restrictions

In [ ]:
turn_restrictions.to_csv(fp.parent/'osm_turn_restrictions.csv',index=False)